In [ ]:
# Week 5 - Regression Modeling
Week Five - August 22, 2022


Reminder, the author has all of the code available on [Github](https://github.com/joelgrus/data-science-from-scratch)

The data set is included in this repo, but sourced from [Kaggle](https://www.kaggle.com/datasets/bagavathypriya/spam-ham-dataset)

## Chapter 14 - Linear Regression

## Chapter 15 - Linear Regression

## Chapter 16 - Linear Regression